# Схемы разделения секрета

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

---

In [3]:
# Генерация простых чисел и КГПСЧ
from Crypto.Util.number import getStrongPrime, getPrime
from Crypto.Random import random

# Утилитки
from functools import reduce

In [4]:
def prod(lst):
    """
    Произведение всех элементов списка.
    """
    
    return reduce((lambda x, y: x * y), lst)

----

In [5]:
class GenericParticipant(object):
    def __init__(self, x, y, t):
        """
        Инициализация участника разделения секрета.
        Сохраняются х, у и t.
        """
        
        self._x = x
        self._y = y
        self._t = t
        
        trace('[GenericParticipant]', f'{x = }, {y = }')
        self._verify()
        
        
    def __eq__(self, other):
        """
        Сравнение на равенство. Сравниваю по х - достаточно.
        """
        
        return self.x == other.x
        
        
    @property
    def x(self):
        """
        Получение х.
        """
        
        return self._x
    
    
    @property
    def y(self):
        """
        Получение у.
        """
        
        return self._y
        
        
    @property
    def equation(self):
        """
        Получение соответствующего участнику уравнения.
        """
        
        return [1] + [self._x ^ i for i in range(1, self._t)], self._y
    
    
    def _verify(self):
        pass

In [6]:
class GenericDealer(object):
    def __init__(self, t, n, field):
        """
        Инициализация дилера для разделения секрета.
        Генерация простого числа, поля и ключа.
        """
        
        self._field = field
        self._t = t
        self._n = n
        self._key = self._field.random_element()
        
        self._coefs = []
        self._values = []
        
        trace('[GenericDealer]', f't = {self._t}, n = {self._n}')
        trace('[GenericDealer]', self._field)
        trace('[GenericDealer]', f'key = {self._key}')
    
    
    @property
    def field(self):
        """
        Получение поля.
        """
        
        return self._field
    
    
    @property
    def key(self):
        """
        Получение ключа для проверки
        """
        
        return self._key
    
    
    def _prepare_coefficients(self):
        self._coefs  = [self._key] + [self._field.random_element() for _ in range(self._t - 1)]
        
        self._values = [sum([c * (x ^ i) for i, c in enumerate(self._coefs)]) 
                        for x in range(1, self._n + 1)]

----

In [7]:
def restore_secret_by_equation_system(field, participants: list[GenericParticipant]):
    """
    Восстановление секрета путем решения системы уравнений.
    """
    
    equations = [p.equation for p in participants]
    M = Matrix(field, [eq[0] for eq in equations])
    v = vector(field, (eq[1] for eq in equations))
    
    return M.solve_right(v)[0]  # Нас интересует только первая переменная,
                                # так как именно она является ключом

In [8]:
def restore_secret_by_lagrange_interpolation(participants: list[GenericParticipant]):
    """
    Восстановление секрета путем интерполяции Лагранжа.
    """
    
    return sum([p1.y * prod([p2.x * (p2.x - p1.x).inverse_of_unit() 
                             for p2 in participants if p1 != p2])
                for p1 in participants])

----

## Схема Шамира
![Shamir scheme](./images/Shamir_1_5.png)
![Shamir scheme](./images/Shamir_2_5.png)

In [9]:
class ShamirParticipant(GenericParticipant):
    def __init__(self, x, y, t):
        """
        Тут в целом все повторяет общий случай.
        """
        
        trace('[ShamirParticipant]', f'Initializing superclass...')
        super(ShamirParticipant, self).__init__(x, y, t)

In [10]:
class ShamirDealer(GenericDealer):
    def __init__(self, t, n):
        """
        Тут в целом все повторяет общий случай.
        """
        
        trace('[ShamirDealer]', f'Initializing superclass...')
        super(ShamirDealer, self).__init__(t, n, GF(getStrongPrime(512)))
        
        
    def split_key(self):
        """
        Разделение секрета между участниками.
        """
        
        self._prepare_coefficients()
        return [ShamirParticipant(self._field.gen() + i, self._values[i], self._t) 
                for i in range(self._n)]

----

In [11]:
t, n = 3, 5

In [12]:
d = ShamirDealer(t, n)

[TRACE] [ShamirDealer] Initializing superclass...
[TRACE] [GenericDealer] t = 3, n = 5
[TRACE] [GenericDealer] Finite Field of size 9778393920576867982535082311225726265560446420199908331972011468402413378132141050255094630791428833198702738213652692814796997564048504240340046027664521
[TRACE] [GenericDealer] key = 2213008301928449335740441386821284495531294779844536250076400954514382313540034021437361552052214173416757055100779143379146076181278977001942155686439456


In [13]:
participants = d.split_key()

[TRACE] [ShamirParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 1, y = 7016032690566348604141970484309753752929082449333699476027373187671808794629047244800616975815902264176711347293708023622834725130372624063441009124493253
[TRACE] [ShamirParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 2, y = 6170148588060527141866417490972948204196899113131905163707440896301890664974166107644276003911438327961490508636363501603356982715766963131734274921057144
[TRACE] [ShamirParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 3, y = 9453749914987852931448864718036594114895191191439061645088615548807041302707531660223433267130251197969797277342398270135509846501510498447161999103795650
[TRACE] [ShamirParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 4, y = 7088442750771457990354229854274965219463512264055260588198885676784847329697002852282994134680912041002928915198159636404496318923554725769384135645044250


In [14]:
restored_key = restore_secret_by_equation_system(d.field, 
                                                 random.sample(participants, t))
print(f'Restored key = {restored_key}')
print(f'Restored correctly: {restored_key == d.key}')

Restored key = 2213008301928449335740441386821284495531294779844536250076400954514382313540034021437361552052214173416757055100779143379146076181278977001942155686439456
Restored correctly: True


In [15]:
false_key = restore_secret_by_equation_system(d.field, 
                                              random.sample(participants, t - 1))
print(f'Restored key = {false_key}')
print(f'Restored correctly: {false_key == d.key}')

Restored key = 6771277487060169772197686998095754535629781553390556483785793696471209843606977033789656033686839835671699625561461478513753431671329312240155543452385329
Restored correctly: False


In [16]:
restored_key = restore_secret_by_lagrange_interpolation(random.sample(participants, t))
print(f'Restored key = {restored_key}')
print(f'Restored correctly: {restored_key == d.key}')

Restored key = 2213008301928449335740441386821284495531294779844536250076400954514382313540034021437361552052214173416757055100779143379146076181278977001942155686439456
Restored correctly: True


In [17]:
false_key = restore_secret_by_lagrange_interpolation(random.sample(participants, t - 1))
print(f'Restored key = {false_key}')
print(f'Restored correctly: {false_key == d.key}')

Restored key = 3732431363639022814559523257246107842230790371026542994646198535166658156895681692221459712597089394168404578587673255090681861344629088748013284941754747
Restored correctly: False


---
## Схема Фельдмана
![Feldman scheme](./images/Feldman_5.png)

In [18]:
class FeldmanParticipant(GenericParticipant):
    def __init__(self, x, y, t, g, check_values):
        """
        Тут в дополнение сохраняется генератор и проверочные значения.
        """
        
        self._g = g
        self._check_values = check_values
        
        trace('[FeldmanParticipant]', f'Initializing superclass...')
        super(FeldmanParticipant, self).__init__(x, y, t)
        
        
    def _verify(self):
        expected_value = self._g ^ self.y
        value_to_check = prod([cv ^ (self.x ^ i) for i, cv in enumerate(self._check_values)])
        
        if value_to_check != expected_value:
            raise ValueError(f'Wrong value given to the participant with x = {self.x} and y = {self.y}')
            
        trace('[FeldmanParticipant]', 'Value verified')

In [19]:
class FeldmanDealer(GenericDealer):
    def __init__(self, t, n):
        """
        В дополнение генерируются p, q, g.
        """
        
        self._p, self._q, self._g = FeldmanDealer._generate_parameters()
        
        trace('[FeldmanDealer]', f'Initializing superclass...')
        super(FeldmanDealer, self).__init__(t, n, GF(self._q))
        
        trace('[FeldmanDealer]', f'p = {self._p}')
        trace('[FeldmanDealer]', f'q = {self._q}')
        trace('[FeldmanDealer]', f'g = {self._g}')
        
        
    def split_key(self):
        """
        Разделение секрета между участниками.
        """
        
        self._prepare_coefficients()
        check_values = [self._g ^ c for c in self._coefs]
        
        return [FeldmanParticipant(self._field.gen() + i, self._values[i], self._t, self._g, check_values) 
                for i in range(self._n)]
    
    
    @staticmethod
    def _generate_parameters():
        def _generate_special_group_element(p, q):
            #
            # Сгенерирую число, взаимно простое с p
            #

            tmp = random.getrandbits(P_LENGTH)
            while gcd(tmp, p) != 1:
                tmp = random.getrandbits(P_LENGTH)

            #
            # По малой теореме Ферма такой g будет иметь порядок q
            #

            F = GF(p)
            return F(tmp) ^ ((p - 1) // q)
        
        Q_LENGTH = 160
        P_LENGTH = 1024
        
        q = getPrime(Q_LENGTH)
        p = random.getrandbits(P_LENGTH - Q_LENGTH) * q + 1
        
        while p not in Primes():
            p = random.getrandbits(P_LENGTH - Q_LENGTH) * q + 1
        
        return p, q, _generate_special_group_element(p, q)

----

In [20]:
d = FeldmanDealer(t, n)

[TRACE] [FeldmanDealer] Initializing superclass...
[TRACE] [GenericDealer] t = 3, n = 5
[TRACE] [GenericDealer] Finite Field of size 844175308291386721997669056681453504624159594643
[TRACE] [GenericDealer] key = 310582981463741321283155758674130692920414193319
[TRACE] [FeldmanDealer] p = 5342564523473778076097614045464127961510971451588151575366422381820584049986294446856639169693224786723900195830998509742816049361392999985848602081892923357831960611158022714024808468041034911651476824591629640732989232070602028731658691056659872235479768967924044832294228914870955896800626544374397337061
[TRACE] [FeldmanDealer] q = 844175308291386721997669056681453504624159594643
[TRACE] [FeldmanDealer] g = 2271865053372486842040576747804357694462665707246798419083768291719871678655362746863507020456061317773452115536881700053107502494718255442567538412408929543151003412958116370760002387415394005711520776621397436644612941675348897419196579160691950460793263708299828472466162768604526941475846360823

In [21]:
participants = d.split_key()

[TRACE] [FeldmanParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 1, y = 726879750193333882049191461651633308158381400349
[TRACE] [FeldmanParticipant] Value verified
[TRACE] [FeldmanParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 2, y = 711663307822942861249458636064448110588736462375
[TRACE] [FeldmanParticipant] Value verified
[TRACE] [FeldmanParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 3, y = 264933654352568258883957281912575100211479379397
[TRACE] [FeldmanParticipant] Value verified
[TRACE] [FeldmanParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 4, y = 230866098073596796950356455877467781650769746058
[TRACE] [FeldmanParticipant] Value verified
[TRACE] [FeldmanParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 5, y = 609460638986028475448656157959126154906607562358
[TRACE] [FeldmanParticipant] Value verified


In [22]:
restored_key = restore_secret_by_equation_system(d.field, 
                                                 random.sample(participants, t))
print(f'Restored key = {restored_key}')
print(f'Restored correctly: {restored_key == d.key}')

Restored key = 310582981463741321283155758674130692920414193319
Restored correctly: True


In [23]:
false_key = restore_secret_by_equation_system(d.field, 
                                              random.sample(participants, t - 1))
print(f'Restored key = {false_key}')
print(f'Restored correctly: {false_key == d.key}')

Restored key = 113677489822329971634139494839708907507672816182
Restored correctly: False


In [24]:
restored_key = restore_secret_by_lagrange_interpolation(random.sample(participants, t))
print(f'Restored key = {restored_key}')
print(f'Restored correctly: {restored_key == d.key}')

Restored key = 310582981463741321283155758674130692920414193319
Restored correctly: True


In [25]:
false_key = restore_secret_by_lagrange_interpolation(random.sample(participants, t - 1))
print(f'Restored key = {false_key}')
print(f'Restored correctly: {false_key == d.key}')

Restored key = 367136323189482644684759760017897055893608279414
Restored correctly: False


----
## Схема Педерсена
![Pedersen scheme](./images/Pedersen_5.png)

In [26]:
class PedersenParticipant(GenericParticipant):
    def __init__(self, i, u, w, g, h, t, check_values):
        """
        Инициализация участника разделения секрета.
        Сохраняются проверочные значения и элементы группы GF*(p)
        """
        
        self._g, self._h = g, h
        self._w = w
        self._check_values = check_values
        
        trace('[PedersenParticipant]', f'Initializing superclass...')
        super(PedersenParticipant, self).__init__(i, u, t)
        
    
    def _verify(self):
        expected_value = (self._g ^ self.y) * (self._h ^ self._w)
        value_to_check = prod([cv ^ (self.x ^ i) for i, cv in enumerate(self._check_values)])
        
        if value_to_check != expected_value:
            raise ValueError(f'Wrong value given to the participant with i = {self.x}')
            
        trace('[PedersenParticipant]', 'Value verified')

In [27]:
class PedersenDealer(GenericDealer):
    def __init__(self, t, n):
        """
        В дополнение к общему случаю генерируются 4 параметра: p, q, g и h.
        """
        
        self._p, self._q, self._g, self._h = PedersenDealer._generate_parameters()
        self._gammas = []
        
        trace('[PedersenDealer]', f'Initializing superclass...')
        super(PedersenDealer, self).__init__(t, n, GF(self._q))
        
        trace('[PedersenDealer]', f'p = {self._p}')
        trace('[PedersenDealer]', f'q = {self._q}')
        trace('[PedersenDealer]', f'g = {self._g}')
        trace('[PedersenDealer]', f'h = {self._h}')
        
        
    def split_key(self):
        """
        Разделение секрета между участниками.
        """
        
        self._prepare_coefficients()
        self._gammas = [self._field.random_element() for _ in range(t)]
        
        check_values = [(self._g ^ d) * (self._h ^ g)
                        for d, g in zip(self._coefs, self._gammas)]
        
        return [PedersenParticipant(self._field.gen() + i, u, self._w(i + 1),
                                    self._g, self._h, self._t, check_values) 
                for i, u in enumerate(self._values)]
    
    
    def _w(self, z):
        return sum([g * (z ^ i) for i, g in enumerate(self._gammas)])
    
    
    @staticmethod
    def _generate_parameters():
        def _generate_special_group_element(p, q):
            #
            # Сгенерирую число, взаимно простое с p
            #

            tmp = random.getrandbits(P_LENGTH)
            while gcd(tmp, p) != 1:
                tmp = random.getrandbits(P_LENGTH)

            #
            # По малой теореме Ферма такой g будет иметь порядок q
            #

            F = GF(p)
            return F(tmp) ^ ((p - 1) // q)
        
        Q_LENGTH = 160
        P_LENGTH = 1024
        
        q = getPrime(Q_LENGTH)
        p = random.getrandbits(P_LENGTH - Q_LENGTH) * q + 1
        
        while p not in Primes():
            p = random.getrandbits(P_LENGTH - Q_LENGTH) * q + 1
            
        g = _generate_special_group_element(p, q)
        h = _generate_special_group_element(p, q)
        
        while g == h:
            h = _generate_special_group_element(p, q)
            
        return p, q, g, h

-----

In [28]:
d = PedersenDealer(t, n)

[TRACE] [PedersenDealer] Initializing superclass...
[TRACE] [GenericDealer] t = 3, n = 5
[TRACE] [GenericDealer] Finite Field of size 1375548585247874962230554118958243704554375800957
[TRACE] [GenericDealer] key = 979739116694319991991538647863318441211638048739
[TRACE] [PedersenDealer] p = 77258409401018966521064626681456795039325202444663949482223836932198143997162294174177095218623355956813019273364948949980363204259259833902826661933906552196115383977494031231885771580914028676522932428762596522582972444117289605413625253532447022375511610047481158692319506786706535734590546023091627887477
[TRACE] [PedersenDealer] q = 1375548585247874962230554118958243704554375800957
[TRACE] [PedersenDealer] g = 487795838996779580532742175451705190044714935457305409267285600133792412928969066359810376172322520105306094237345884211715509516003233448396612359906476411441572187389246147882774659201016186387677422278846634539341443814248827977101245900487496280455073905393241791289791405546915024595866

In [29]:
participants = d.split_key()

[TRACE] [PedersenParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 1, y = 1342672357092746945230589248178869187182977247320
[TRACE] [PedersenParticipant] Value verified
[TRACE] [PedersenParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 2, y = 488979298767849806103744885750058024870431104352
[TRACE] [PedersenParticipant] Value verified
[TRACE] [PedersenParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 3, y = 1169757112215378499072113798493372363382751221749
[TRACE] [PedersenParticipant] Value verified
[TRACE] [PedersenParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 4, y = 633908626939583099674587748492324793611185997597
[TRACE] [PedersenParticipant] Value verified
[TRACE] [PedersenParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 5, y = 256982428188338570141720854705159020110111232853
[TRACE] [PedersenParticipant] Value verified


In [30]:
restored_key = restore_secret_by_equation_system(d.field, 
                                                 random.sample(participants, t))
print(f'Restored key = {restored_key}')
print(f'Restored correctly: {restored_key == d.key}')

Restored key = 979739116694319991991538647863318441211638048739
Restored correctly: True


In [31]:
false_key = restore_secret_by_equation_system(d.field, 
                                              random.sample(participants, t - 1))
print(f'Restored key = {false_key}')
print(f'Restored correctly: {false_key == d.key}')

Restored key = 1163370553008063430237149095217448673737335404136
Restored correctly: False


In [32]:
restored_key = restore_secret_by_lagrange_interpolation(random.sample(participants, t))
print(f'Restored key = {restored_key}')
print(f'Restored correctly: {restored_key == d.key}')

Restored key = 979739116694319991991538647863318441211638048739
Restored correctly: True


In [33]:
false_key = restore_secret_by_lagrange_interpolation(random.sample(participants, t - 1))
print(f'Restored key = {false_key}')
print(f'Restored correctly: {false_key == d.key}')

Restored key = 1163370553008063430237149095217448673737335404136
Restored correctly: False
